In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 15 21:56:16 2021

@author: danso
"""

import numpy as np
import struct
import serial
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from IPython.display import clear_output
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [13]:
# target = np.random.randint(10)
target = np.repeat(np.arange(10), 2)
np.random.shuffle(target)
# print(target)


bigdata = np.zeros([20,3*119,7])

for i in range(20):
    printmd("<span style=\"font-size:3em;\">" + str(target[i]) + "</span>")
    data = []
    with serial.Serial('COM3', 38400, timeout=10) as ser:
        startmess = ser.readline()
        if not startmess:
            raise(Exception("Timed Out"))
        if startmess != b'IMU goes BBBBRRRR\r\n':
            raise(Exception("Start message invalid:\n"+str(startmess)))
        ser.timeout = 0.1
        while True:
            message = ser.read(28)
            if not message:
                raise(Exception("Timed Out"))
            if len(message)<24:
                raise(Exception("Message Too Short:\n"+str(message)))
            values = np.array(struct.unpack('fffffff',message))
            if np.any(np.isnan(values)):
                if np.any(~np.isnan(values)):
                    raise Exception("Could not read data:\n"+str(message))
                else:
                    break
            data.append(values)
    data = np.array(data)
    clear_output(wait=True)
    print(data.shape[0])
    data = np.pad(data,((0, 3*119-data.shape[0],), (0,0)))
    #print(data)
    print(np.mean(data, axis=0))
    bigdata[i] = data

116
[-9.00980266e-02  5.28750754e-02 -3.80994626e-03  2.05096589e+00
 -5.28447512e+00  5.35656157e+00  1.59525084e+05]


In [ ]:
np.set_printoptions(suppress=True)
print(np.mean(bigdata[:, :, :-1],axis=1))

In [14]:
together = (bigdata, target)
now = datetime.now()
name = now.strftime("Dan_collected_%H_%M_%S.pkl")
print(name)
with open("data/"+name, "wb") as fp:
    pickle.dump(together, fp)

Dan_collected_23_53_12.pkl


In [ ]:
temp=bigdata[9][:,(1,2)]
temp1 = np.cumsum(temp,axis=0)
temp2 = np.cumsum(temp1-np.mean(temp1, axis=0),axis=0)
plt.plot(temp2[:, 0], temp2[:, 1])

In [ ]:
print(target)

In [ ]:
with open("data/collected224735.pkl", "rb") as fp:
    x,y = pickle.load(fp)

In [ ]:
print(x.shape)